# 매일경제의 삼성전자 관련 기사 크롤링
http에러때문에 한번에 40페이지정도 크롤링 해오는 것을 추천   
1페이지는 모종의 문제로 2페이지부터 크롤링 시작(1페이지는 따로)

In [26]:
all_article = [] #수집되는 전체 기사가 들어가는 리스트 [['제목','날짜','내용','url'],['제목','날짜','내용','url']...]로 나타내어짐

In [18]:
from urllib.request import urlopen
import bs4
from urllib.parse import quote
import csv
start_num = int(input())
page_num = int(input())

for i in range(start_num,page_num+1): #1페이지에 문제가 있어 2페이지부터 시작해야함 ,40페이지씩 끊는것을 추천
    print(i)
    #매경 검색창에 기간과 기업명을 설정하고 검색한 결과의 페이지
    url = "http://find.mk.co.kr/new/search.php?pageNum="+str(i)+"&cat=&cat1=&media_eco=&pageSize=20&sub=news&dispFlag=OFF&page=news&s_kwd=%BB%EF%BC%BA%C0%FC%C0%DA&s_page=news&go_page=&ord=1&ord1=1&ord2=0&s_keyword=%BB%EF%BC%BA%C0%FC%C0%DA&period=p_direct&s_i_keyword=%BB%EF%BC%BA%C0%FC%C0%DA&s_author=&y1=2020&m1=06&d1=22&y2=2020&m2=06&d2=25&media%5B1%5D=00001&ord=1&area=ttbd"
    print(url)
    while True:
        #검색결과가 나왔다 안나왔다 할때가 있어서 검색결과가 나온페이지의 태그가 뜰때까지 예외처리로 반복
        try:
            html = urlopen(url)
            bs = bs4.BeautifulSoup(html.read(),"html.parser")
            td = bs.find("td",{"class":"title"})
            a = td.text
            
        except AttributeError:
            print("check")
            span = bs.findAll("span",{"class":"art_tit"})
            print(span)
            for i in span:
                a = i.find("a")
                link = a["href"]
                html = urlopen(link)
                bs = bs4.BeautifulSoup(html.read(),"html.parser")
                date = bs.find("li",{"class":"lasttime"})
                if int(date.text[16:18]) >17: # 18시 이후 등록 기사 제거
                    continue
                empty =[]
                try:
                    h1 = bs.find("h1",{"class":"top_title"})
                    empty.append(h1.text)#제목
                except AttributeError:
                    print("오류기사")
                    pass
            
                try:
                    li = bs.find("li",{"class":"lasttime"})
                    empty.append(li.text[5:15]) #날짜  

                except AttributeError:
                    try:
                        li = bs.find("li",{"class":"lasttime1"})
                        empty.append(li.text[5:15]) #날짜
                    except:
                        empty.append("오류기사")
                
                try:
                    div = bs.find("div",{"class":"art_txt"})
                    article = div.get_text().replace('\r',"").replace('\n','').replace('\t','')
                    empty.append(article) #기사
                except AttributeError:
                    empty.append("오류기사 전문")
                    print("오류기사")
                empty.append(link) #링크
                all_article.append(empty)
                if empty[1][0] != '2'or '[표]' in empty[0] or (empty[0] or empty[1]) =="오류기사":
                    all_article.pop()
                
                print(link)
            break
    



1
1
1
http://find.mk.co.kr/new/search.php?pageNum=1&cat=&cat1=&media_eco=&pageSize=20&sub=news&dispFlag=OFF&page=news&s_kwd=%BB%EF%BC%BA%C0%FC%C0%DA&s_page=news&go_page=&ord=1&ord1=1&ord2=0&s_keyword=%BB%EF%BC%BA%C0%FC%C0%DA&period=p_direct&s_i_keyword=%BB%EF%BC%BA%C0%FC%C0%DA&s_author=&y1=2020&m1=06&d1=22&y2=2020&m2=06&d2=25&media%5B1%5D=00001&ord=1&area=ttbd
check
[<span class="art_tit"><a href="https://www.mk.co.kr/news/economy/view/2020/06/653589/" target="_blank"><b>삼성전자</b>가 블록체인 스마트폰을 또 내놓는다고?</a></span>, <span class="art_tit"><a href="https://www.mk.co.kr/news/stock/view/2020/06/653548/" target="_blank">2분기 상장사 실적 뜻밖의 선전?…환율·비용절감 효과 `톡톡`</a></span>, <span class="art_tit"><a href="https://www.mk.co.kr/news/business/view/2020/06/653531/" target="_blank">이재용의 `K칩 동행`…협력사 손잡고 `반도체 초격차` 나선다</a></span>, <span class="art_tit"><a href="https://www.mk.co.kr/news/business/view/2020/06/653530/" target="_blank">日수출규제 1년…국산화 일부 성과냈지만 소부장 갈길 멀어</a></span>, <span class="art_tit"><a href="http

In [19]:
f = open('articles.csv','w',encoding='utf-8-sig')
wr = csv.writer(f)
for i in all_article:
    url = i[3]
    if 'society' in i[3] or 'special' in i[3] or 'home' in i[3] or 'realestate' in i[3] or 'politics' in i[3]: #관계가 없어보이는 
        continue
    else:
        if '삼성전자' in i[0] or '삼성전자' in i[2]:
            if i[0] == i[2][5:len(i[0])+5]: #본문에 제목이 포함돼있는경우 그부분 제거
                i[2] = i[2][5+len(i[0]):]
            wr.writerow(i)

f.close()